# Chinook Business Project

## Introducation and Schema Diagram of Database

We'll be working with a modified version of a database called Chinook. The Chinook database contains information about a fictional digital music shop - kind of like a mini-iTunes store.

The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables.

The Chinook database is provided as a SQLite database file called `chinook.db`. A copy of the database schema is below:

<img src="chinook-schema.svg" width="600" height="300">

We are going to use SQL queries to analyse the following:

1. The best selling genre in USA
2. Performance of the sales representative
3. Sales by country
4. Purchase of albums vs purchase of individual tracks
5. Which artist is used in the most playlists?
6. How many tracks have been purchased vs not purchased?
7. Is the range of tracks in the store reflective of their sales popularity?
8. Do protected vs non-protected media types have an effect on popularity?

We'll use the following code to connect our Jupyter Notebook to our database file:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
sql

 * sqlite:///chinook.db


'Connected: @chinook.db'

## Overview Database 

Let's start by getting familiar with our data.

In [3]:
%%sql

SELECT name, type
FROM sqlite_master
WHERE type IN ('table','view');

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


We can see there are 11 tables in the database and every table can directly or indirectly connect each other, which was found on Database Schema Diagram.

## Analysis Data

### 1. genres sell the most tracks in the USA

The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| Artist Name | Genre    |
| ---         | ---      |
| Regal       | Hip-Hop  | 
| Red Tone    | Punk     |
|Meteor and the Girls|Pop|
|Slim Jim Bites| Blues   |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.


In order to find which genres sell the best in the USA, we need to use `track, invoice_line, invoice, genre,customer` tables. We should limit country is USA.

In [4]:
%%sql

SELECT 
    g.name 'genre_name',
    SUM(il.quantity) 'tracks_sold'
FROM track t
INNER JOIN genre 'g' ON g.genre_id = t.genre_id
INNER JOIN invoice_line 'il' ON il.track_id = t.track_id
INNER JOIN invoice 'i' ON i.invoice_id = il.invoice_id
INNER JOIN customer 'c' ON c.customer_id = i.customer_id
WHERE c.country ='USA'
GROUP BY 1
ORDER BY 2 DESC;


 * sqlite:///chinook.db
Done.


genre_name,tracks_sold
Rock,561
Alternative & Punk,130
Metal,124
R&B/Soul,53
Blues,36
Alternative,35
Pop,22
Latin,22
Hip Hop/Rap,20
Jazz,14


We will update sql query to returns each genre, with the number of tracks sold in the USA:

* in absolute numbers
* in percentages.

In [5]:
%%sql

WITH usa_genre_tracks AS (
    SELECT 
        g.name 'genre_name',
        SUM(il.quantity) 'tracks_sold'
    FROM track t
    INNER JOIN genre 'g' ON g.genre_id = t.genre_id
    INNER JOIN invoice_line 'il' ON il.track_id = t.track_id
    INNER JOIN invoice 'i' ON i.invoice_id = il.invoice_id
    INNER JOIN customer 'c' ON c.customer_id = i.customer_id
    WHERE c.country ='USA'
    GROUP BY 1
    ORDER BY 2 DESC
)
SELECT 
    ugt.genre_name,
    ugt.tracks_sold,
    CAST(ugt.tracks_sold AS FLOAT) * 100 / (SELECT CAST(SUM(tracks_sold) AS FLOAT)
                                            FROM usa_genre_tracks) "percentage_sold"
FROM usa_genre_tracks 'ugt';



 * sqlite:///chinook.db
Done.


genre_name,tracks_sold,percentage_sold
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.4253092293054235
Alternative,35,3.330161750713606
Pop,22,2.093244529019981
Latin,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


From analysing the most commonly sold genres in USA , its found that Rock music tracks are most liked by the music lovers which is around 53.378 % followed by Alternative &Punk,Metal,R&B/Soul in that order.Blues genres is in 5 th position whereas pop and Hip/Hop are in 8 th and 9 th position respectively.

Based on the sales of tracks from their genres,we could recommend the following 3 artists:

1. Red Tone, genre-Punk
2. Slim Jim Bites , genre-Blues
3. Meteor and the Girls, genre- Pop

### 2. Analysing performance of Sales Agents

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

To do this, we need `employee, customer`, and `invoice` tables (refer schema)

In [6]:
%%sql
SELECT 
    e.first_name || ' ' || e.last_name 'Employee Name',
    ROUND(SUM(i.total),2) 'Total_amount of Sales',
    e.birthdate 'Birthdate',
    e.hire_date 'Hire Date',
    e.country 'Country'
FROM employee 'e'
INNER JOIN customer 'c' ON c.support_rep_id = e.employee_id
INNER JOIN invoice 'i' ON i.customer_id = c.customer_id
GROUP BY 1
ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


Employee Name,Total_amount of Sales,Birthdate,Hire Date,Country
Jane Peacock,1731.51,1973-08-29 00:00:00,2017-04-01 00:00:00,Canada
Margaret Park,1584.0,1947-09-19 00:00:00,2017-05-03 00:00:00,Canada
Steve Johnson,1393.92,1965-03-03 00:00:00,2017-10-17 00:00:00,Canada


The store has 3 Sales Support Agents, all of them from Canada. All agents seem to have the same amount of experience with the company as seen in their hiring dates. The difference is sales roughly corresponds with the difference in hiring dates.

### 3. Analyzing Sales by Country

In particular, we have been directed to calculate data, for each country, on the:

* total number of customers
* total value of sales
* average value of sales per customer
* average order value

Because there are a number of countries with only one customer, we should group these customers as "Other" in your analysis. 

In [7]:
%%sql
SELECT
    CASE
        WHEN(
            SELECT COUNT(*)
            FROM customer
            WHERE country = c.country
        )=1 THEN 'Other'
        ELSE c.country
        END country,
    COUNT(DISTINCT(c.customer_id)) AS Total_Customers,
    COUNT(i.invoice_id) AS Total_Orders,
    ROUND(SUM(i.total),2) total_sales
FROM invoice i
INNER JOIN customer c ON c.customer_id = i.customer_id
GROUP BY 1;

 * sqlite:///chinook.db
Done.


country,Total_Customers,Total_Orders,total_sales
Brazil,5,61,427.68
Canada,8,76,535.59
Czech Republic,2,30,273.24
France,5,50,389.07
Germany,4,41,334.62
India,2,21,183.15
Other,15,147,1094.94
Portugal,2,29,185.13
USA,13,131,1040.49
United Kingdom,3,28,245.52


In [8]:
%%sql
WITH country_information AS(
    SELECT
        CASE
            WHEN(
                SELECT COUNT(*)
                FROM customer
                WHERE country = c.country
            )=1 THEN 'Other'
            ELSE c.country
            END country,
        COUNT(DISTINCT(c.customer_id)) AS 'Total_Customers',
        COUNT(i.invoice_id) AS 'Total_Orders',
        ROUND(SUM(i.total),2) 'Total_Sales'
    FROM invoice 'i'
    INNER JOIN customer 'c' ON c.customer_id = i.customer_id
    GROUP BY 1
)
SELECT 
    country 'Country',
    Total_Customers,
    Total_Sales,
    Avg_Sales_per_Customer,
    Avg_Order_Value
FROM(
    SELECT 
        ci.*,
        ROUND(CAST(Total_Sales AS FLOAT) / Total_Customers, 2) 'Avg_Sales_per_Customer',
        ROUND(CAST(Total_Sales AS FLOAT) / Total_Orders, 2) 'Avg_Order_Value',
        CASE 
            WHEN ci.country = 'Other' THEN 1
            ELSE 0
            END AS sort
    FROM country_information 'ci'
    GROUP BY country
    ORDER BY sort ASC, Total_Customers DESC
    );


 * sqlite:///chinook.db
Done.


Country,Total_Customers,Total_Sales,Avg_Sales_per_Customer,Avg_Order_Value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
United Kingdom,3,245.52,81.84,8.77
Czech Republic,2,273.24,136.62,9.11
India,2,183.15,91.58,8.72
Portugal,2,185.13,92.57,6.38
Other,15,1094.94,73.0,7.45


Based on the data, there may be opportunity in the following countries:

* Czech Republic
* United Kingdom
* India

It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

### 4. Albums vs Individual Tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

* purchase a whole album
* purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In this instance, we have two edge cases to consider:

* Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
* Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.

To do this, we need the `invoice_line`, `album`, and `track` tables (refer schema). We are going to check whether each invoice has all the tracks from an album.

In [9]:
%%sql
SELECT * FROM invoice_line
LIMIT 5;

 * sqlite:///chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1


In [10]:
%%sql
SELECT * FROM track
LIMIT 5;

 * sqlite:///chinook.db
Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [11]:
%%sql
SELECT * FROM album
LIMIT 5;

 * sqlite:///chinook.db
Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [12]:
%%sql
WITH invoice_info AS 
    (
    SELECT 
        il.invoice_id,
        t.album_id,
        COUNT(il.track_id) num_tracks
    FROM invoice_line il
    LEFT JOIN track t on t.track_id = il.track_id
    GROUP BY invoice_id, album_id
    ),
    
album_info AS
    (
    SELECT 
        album_id,
        COUNT(track_id) num_tracks
    FROM track
    GROUP BY 1
    )
    
SELECT 
    CASE
        WHEN (SELECT num_tracks FROM album_info WHERE album_id = iv.album_id) = iv.num_tracks AND al.num_tracks > 1
        THEN 'Album'
        ELSE 'Not album'
        END AS album_purchase,
    COUNT(DISTINCT invoice_id) num_invoices,
    ROUND(CAST(COUNT(DISTINCT invoice_id) as float) / (SELECT COUNT(DISTINCT invoice_id) FROM invoice_info) * 100, 2) percent_invoices
FROM invoice_info iv
LEFT JOIN album_info al on al.album_id = iv.album_id
GROUP BY album_purchase
LIMIT 20;

 * sqlite:///chinook.db
Done.


album_purchase,num_invoices,percent_invoices
Album,111,18.08
Not album,503,81.92


Purchases of full albums account for about 18% of total sales. Based on this, I would recommend to continue selling full albums, as this accounts for 1/5 of total purchases. Removing whole albums might signficantly cut revene as a result.

### 5. Which artist is used in the most playlists?


In [13]:
%%sql
SELECT ar.name "artist", count(p.playlist_id) "count" ,p.name "playlist", g.name "genre"
FROM  playlist_track pt
LEFT JOIN track t  ON pt.track_id = t.track_id
LEFT JOIN album al ON al.album_id = t.album_id
LEFT JOIN artist ar ON ar.artist_id = al.artist_id
LEFT JOIN genre g ON t.genre_id = g.genre_id
LEFT JOIN playlist p ON p.playlist_id = pt.playlist_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;


 * sqlite:///chinook.db
Done.


artist,count,playlist,genre
Iron Maiden,516,Music,Blues
U2,333,Music,Pop
Metallica,296,Music,Metal
Led Zeppelin,252,Music,Rock
Deep Purple,226,Music,Rock
Lost,184,TV Shows,TV Shows
Pearl Jam,177,Music,Alternative & Punk
Faith No More,145,Music,Alternative & Punk
Eric Clapton,145,Music,Blues
Lenny Kravitz,143,Music,Metal


We can find that Iron Maiden has been used in most playlists.

### 6. How many tracks have been purchased vs not purchased?

The management at Chinook wants to understand what share of tracks in the store are bought by customers to be able to manage the inventory better.

To do this, we need the `track` and `invoice_line` tables (refer schema). `invoice_line` contains the tracks that are sold while track contains all the tracks in the store.

In [14]:
%%sql

WITH invoiced AS 
    (
        SELECT  track_id  FROM invoice_line
    )
        
SELECT COUNT(DISTINCT p.track_id) as tracks_purchased, 
(
    SELECT COUNT(DISTINCT track_id)
    FROM track 
    WHERE track_id NOT IN (SELECT track_id FROM invoiced)
) AS tracks_not_purchased
FROM invoiced p  
LEFT JOIN track t ON t.track_id = p.track_id


 * sqlite:///chinook.db
Done.


tracks_purchased,tracks_not_purchased
1806,1697


In [15]:
%%sql
SELECT
    COUNT(*)
FROM track

 * sqlite:///chinook.db
Done.


COUNT(*)
3503


Out of 3503 tracks available in Chinook's store, 1806 have been purchased, and 1697 have not been. Almost half of the tracks available have yet to be purchased by any customer. Chinook may want to remove some of these tracks from its inventory.

### 7. Is the range of tracks in the store reflective of their sales popularity?


In [16]:
%%sql
SELECT
    g.name genre,
    COUNT(DISTINCT t.track_id) tracks_available,
    ROUND(CAST(COUNT(DISTINCT t.track_id) as float) / CAST((SELECT COUNT(*) FROM track) as float) * 100, 2) percentage_tracks_avail,
    COUNT(DISTINCT il.track_id) track_has_sold,
    ROUND(CAST(COUNT(DISTINCT il.track_id) as float) / CAST(COUNT(DISTINCT t.track_id) as float) * 100, 2) as percent_has_sold,
    COUNT(il.track_id) number_sold,
    ROUND(CAST(COUNT(il.track_id) as float) / CAST((SELECT COUNT(track_id) FROM invoice_line) as float) * 100, 2) percentage_total_sales
FROM track t
LEFT JOIN genre g ON g.genre_id = t.genre_id
LEFT JOIN invoice_line il ON il.track_id = t.track_id
GROUP BY g.name
ORDER BY 6 DESC

 * sqlite:///chinook.db
Done.


genre,tracks_available,percentage_tracks_avail,track_has_sold,percent_has_sold,number_sold,percentage_total_sales
Rock,1297,37.03,915,70.55,2635,55.39
Metal,374,10.68,238,63.64,619,13.01
Alternative & Punk,332,9.48,176,53.01,492,10.34
Latin,579,16.53,119,20.55,167,3.51
R&B/Soul,61,1.74,55,90.16,159,3.34
Blues,81,2.31,56,69.14,124,2.61
Jazz,130,3.71,61,46.92,121,2.54
Alternative,40,1.14,34,85.0,117,2.46
Easy Listening,24,0.69,24,100.0,74,1.56
Pop,48,1.37,25,52.08,63,1.32


In general, the range of tracks available for sale was reflective of that genre's popularity.

For example, the 3 top selling genres had a larger variety of tracks available than other genres

* Rock - 55% of total sales, 1297 tracks available (37% of tracks available for purchase)
* Metal - 13% of total sales, 374 tracks available (10% of tracks available for purchase)
* Alternative & Punk - 10% of total sales, 332 tracks available (9% of tracks available for purchase) Within the Rock genre, 70% of tracks available have sold. The same is true of 63% of Metal tracks and 53% of Alternative & Punk tracks. Rock was by far the most represented genre in the top ten selling artists, albums, and tracks, followed by metal. Chinook may want to continue to expand its music offerings in Rock and possibly Metal, or advertize its current catalog in these genres, as these are the best selling genres in the store. A relatively high percentage of Rock and Metal tracks sell, so additional purchases in these areas may be low-risk opportunities to increase business.

One noteable exception to this is the Latin genre, which represented only 3% of total sales but had 579 tracks available (16% of total tracks available for purchase). Moreover, only 20% of Latin tracks available at Chinook have ever been sold. As a result, Chinook may want to consider dropping some of these tracks from its collection.

There are 7 genres that have not sold any tracks. 4 additional genres account for less than .5% of sales each. Chinook may want to consider eliminating these offerings as well.

### 8. Protected vs Non-protected media types

To do this, we need `media_type, track`, and `invoice_line` tables (refer schema). From the previous query we know that there are 1806 distinct tracks sold by the store.

In [17]:
%%sql
SELECT * 
FROM media_type;

 * sqlite:///chinook.db
Done.


media_type_id,name
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file
4,Purchased AAC audio file
5,AAC audio file


In [18]:
%%sql
WITH protected_media AS(
    SELECT 
        CASE
            WHEN m.name LIKE '%protected%'
            THEN 'Yes'
            ELSE 'No'
            END AS protected,
        t.track_id
    FROM media_type m
    INNER JOIN track t ON t.media_type_id = m.media_type_id
    )
SELECT 
    pm.protected,
    COUNT(DISTINCT(il.track_id)) tracks_sold
FROM invoice_line il
LEFT JOIN protected_media pm ON il.track_id = pm.track_id
GROUP BY 1;

 * sqlite:///chinook.db
Done.


protected,tracks_sold
No,1652
Yes,154


Out of all the unique tracks sold (1806 tracks) by the store, ~91% comprises of unprotected media.

## Conclusion
Some of the findings are :

* From analysing the most commonly sold genres in USA , its found that Rock music tracks are most liked by the music lovers which is around 53.378 % followed by Alternative &Punk,Metal,R&B/Soul in that order.
* The chinook store should continue to buy single tracks from record companies since the percentage of full album purchased is only about 18.56 % where as single track purchase is 81.43 % .
* Tracks belonging to Rock genre is more popular than other genres and the store has about 37.03 % of tracks in Rock genre .
* Non protected media types more popular with a track sales of 90.71 % than protected media types.